# Gap Framework - Computer Vision / CNN

In this tutorial, we will show you how to prepare a dataset for a convolutional neural network. We will do the following:

1. Preprocess a collection of images of fruits from the Kaggle Fruits-360 dataset into Machine Learning ready data.
2. Store the Machine Learning ready data into a repository.
3. Create a batch feeder.
4. Create a CNN.
5. Retreive the Machine Learning ready data.
6. Train the CNN with our Machine Learning ready data.

In [1]:
# Let's go the directory of the Gap Framework
import os
os.chdir("../")
!cd

C:\Users\'\Desktop\Gap-ml


### Setup

Let's start by importing the Gap <b style='color:saddlebrown'>vision</b> module

In [2]:
# import the Gap Vision module
from vision import Image, Images

Let's go to a respository of images for classifying types of fruits. We will use this repository for image preprocessing for computer vision.

The training and test datasets are under the corresponding subfolders Training and Test. Each subfolder under Training (and Test) is named according to the type of fruit (e.g., Apple) and optionally followed by a variety (e.g., Red Delicious).

Let's take a look at the subfolders and see how many different classes of fruits are in our training set (i.e., 76).

In [3]:
os.chdir("../FruitMaps/fruits/fruits-360/Training")

# Let's get a list of all the subfolders of collections of fruits
labels = os.listdir()
print("Number of Labels:", len(labels))
print(labels)

Number of Labels: 76
['Apple Braeburn', 'Apple Golden 1', 'Apple Golden 2', 'Apple Golden 3', 'Apple Granny Smith', 'Apple Red 1', 'Apple Red 2', 'Apple Red 3', 'Apple Red Delicious', 'Apple Red Yellow', 'Apricot', 'Avocado', 'Avocado ripe', 'Banana', 'Banana Red', 'Cactus fruit', 'Cantaloupe 1', 'Cantaloupe 2', 'Carambula', 'Cherry 1', 'Cherry 2', 'Cherry Rainier', 'Cherry Wax Black', 'Cherry Wax Red', 'Cherry Wax Yellow', 'Clementine', 'Cocos', 'Dates', 'fruits.h5', 'Granadilla', 'Grape Pink', 'Grape White', 'Grape White 2', 'Grapefruit Pink', 'Grapefruit White', 'Guava', 'Huckleberry', 'Kaki', 'Kiwi', 'Kumquats', 'Lemon', 'Lemon Meyer', 'Limes', 'Lychee', 'Mandarine', 'Mango', 'Maracuja', 'Melon Piel de Sapo', 'Mulberry', 'Nectarine', 'Orange', 'Papaya', 'Passion Fruit', 'Peach', 'Peach Flat', 'Pear', 'Pear Abate', 'Pear Monster', 'Pear Williams', 'Pepino', 'Physalis', 'Physalis with Husk', 'Pineapple', 'Pineapple Mini', 'Pitahaya Red', 'Plum', 'Pomegranate', 'Quince', 'Rambutan', '

Let's now look a little closer at the images in the training set. We will dive into the first subfolder (Apple Braeburn).

In [4]:
# Let's get a listing of all the images in the first subfolder.
data = os.listdir(labels[0])
print("Number of Images:", len(data))

Number of Images: 492


We will use openCV to get some basic information on the images. From the shape of the pixel data we see that its a 100x100 pixel image with three channels (i.e., RGB).

In [5]:
# Import the openCV module
import cv2

# We will look at the first image in this first collection.
print(data[0])

# Use openCV to read the image into memory as an uncompressed bitmap
pixels = cv2.imread(labels[0] + '/' + data[0])

# Let's look at the shape of the image.
print(pixels.shape)

0_100.jpg
(100, 100, 3)


Let's look at a few more random images in this subfolder and see if they are all the same size and type.

In [6]:
# Our random selection of images
for index in [ 7, 26, 143 ]:
    print(data[index])

    # Use openCV to read the image into memory as an uncompressed bitmap
    pixels = cv2.imread(labels[0] + '/' + data[index])

    # Let's look at the shape of the image.
    print(pixels.shape)

106_100.jpg
(100, 100, 3)
123_100.jpg
(100, 100, 3)
229_100.jpg
(100, 100, 3)


Okay, the are the same size.

Let's look a different collection of fruits and see if they too are the same size. Let's use the 8th (index 7) subfolder.

In [7]:
# Our random selection of images
for index in [ 7, 26, 143 ]:
    print(data[index])

    # Use openCV to read the image into memory as an uncompressed bitmap
    pixels = cv2.imread(labels[7] + '/' + data[index])

    # Let's look at the shape of the image.
    print(pixels.shape)

106_100.jpg
(100, 100, 3)
123_100.jpg
(100, 100, 3)
229_100.jpg
(100, 100, 3)


### Practice

Let's do a practice run and preprocess one collection of fruit images.

Note, how we specified the subfolder instead of a list for the parameter images. The initializer (constructor) looks at the parameter and if its not a list, but a string it presumes the parameter is a path to a folder with images.

In [8]:
images = Images(labels[0], 0)
print("TIME", images.time)

TIME 4.9764087200164795


Perhaps our images won't need to be as big to train the CNN. Let's take a shot in the dark and say they only need to be 50x50. This will reduce the size of our data by 75%.

In [9]:
images = Images(labels[0], 0, config=['resize=(50,50)'])
print("TIME", images.time)

os.remove('collection.0_100.h5')

TIME 3.5880062580108643


### Prepare the Data

The labels of the fruits are names, but we need integer values to train the CNN. Since all the subfolder names (fruit name+variety) are in the list labels, we will use the index of the list as the labels.

For brevity of time, we will only create machine learning ready data for three of the fruit collections (hence why we commented out the line for doing the entire set of fruits).

In [10]:
# Process all the Collections (subfolders) of Fruits
#images = Images(labels, [l for l in range(len(labels))], config=['resize=(50,50)'], name='fruits')

# For brevity, let's just do three of them
images = Images([labels[0], labels[1], labels[2]], [l for l in range(3)], config=['resize=(50,50)'], name='fruits')

print("TIME:", images.time)

### Batch Generation

In this example, our training and test data are in separate collections. We still want to randomize the order of images in the training data for batch generation. We can still do this, we just set the split to 0.0. In this case, all the training data is used for training batch generation.

In [13]:
# Keep all the images for training, randomize their order
images.split = 0.0, 42

# Let's look at the (internal) _train property and verify that the indices of the images has been randomized.
images._train

[1309,
 228,
 51,
 563,
 501,
 457,
 285,
 209,
 1385,
 1116,
 178,
 1209,
 864,
 65,
 61,
 191,
 447,
 476,
 1034,
 1232,
 54,
 1149,
 407,
 1330,
 1436,
 1466,
 859,
 451,
 919,
 1206,
 569,
 13,
 326,
 1429,
 865,
 696,
 1445,
 318,
 440,
 689,
 1468,
 189,
 778,
 198,
 735,
 704,
 1236,
 541,
 88,
 940,
 1098,
 255,
 775,
 161,
 1130,
 600,
 1287,
 1266,
 740,
 1182,
 393,
 142,
 93,
 1354,
 466,
 592,
 163,
 1458,
 206,
 1433,
 1439,
 928,
 1301,
 747,
 333,
 758,
 727,
 429,
 1372,
 546,
 1327,
 146,
 1247,
 1300,
 350,
 1093,
 1471,
 334,
 946,
 777,
 552,
 1310,
 1140,
 449,
 664,
 114,
 469,
 1462,
 646,
 821,
 548,
 135,
 432,
 1161,
 644,
 435,
 1342,
 1022,
 810,
 1316,
 939,
 292,
 542,
 1469,
 505,
 1454,
 1103,
 538,
 1197,
 877,
 1195,
 817,
 741,
 1382,
 283,
 1043,
 1010,
 186,
 96,
 224,
 313,
 1285,
 327,
 1463,
 1221,
 130,
 788,
 781,
 1220,
 958,
 1083,
 514,
 1133,
 23,
 234,
 1099,
 1396,
 1312,
 1440,
 1474,
 601,
 890,
 323,
 929,
 6,
 539,
 1025,
 365,
 1039

Let's create our mini-barch generator

In [15]:
images.minibatch = 32